In [2]:
using BenchmarkTools  

In [13]:
a = [9 , 2, 8, 3, 4, 8]
juliaSortFunc = sort(a)

6-element Array{Int64,1}:
 2
 3
 4
 8
 8
 9

In [14]:
Julia_list_bench = @benchmark $juliaSortFunc

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.500 ns (0.00% GC)
  median time:      2.800 ns (0.00% GC)
  mean time:        3.051 ns (0.00% GC)
  maximum time:     416.312 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [15]:
using PyCall
import PyCall
py"""
a = sorted([1,2,8,4,5])
"""
pyMergeBI= py"a"

5-element Array{Int64,1}:
 1
 2
 4
 5
 8

In [16]:
py_list_bench = @benchmark $pyMergeBI

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.400 ns (0.00% GC)
  median time:      2.600 ns (0.00% GC)
  mean time:        2.836 ns (0.00% GC)
  maximum time:     417.913 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [4]:
## Cited from: https://github.com/sashang/notebook_of_code/blob/master/julia/merge-sort/merge-sort.jl
function mergesort(x)
    function combine(lhs, rhs)
        j = 1
        k = 1
        result = Float64[]
        while true
            #if left>right,add right, compare next right to left
            if lhs[j] > rhs[k]
                append!(result, [rhs[k]])
                k += 1
                #else add left
            else
                append!(result, [lhs[j]])
                j += 1
            end
            #After comparing if left or right has left elements, append to the list 
            if j == length(lhs) + 1
                append!(result, rhs[k:end])
                break
            end
            if k == length(rhs) + 1
                append!(result, lhs[j:end])
                break
            end
        end
        return result
    end

    l = length(x)
    if l > 2
        #divide the array into two parts from the middle
        left = x[1 : div(l, 2)]
        right = x[div(l, 2) + 1 : end]
        #merge left and right sides
        left_result = mergesort(left)
        right_result = mergesort(right)
        return combine(left_result, right_result)
    #if length < 2, return
    elseif l < 2
        return x
        #else change integer placss
    else
        if x[1] > x[2]
            temp = x[1]
            x[1] = x[2]
            x[2] = temp
        end
        return x
    end
end

question = [3,1,4,2]
result = mergesort(question)
println("question = $question, result = $result")
question = [5,1,3,9,0]
result = mergesort(question)
println("question = $question, result = $result")
julia_mergeSort_bench = @benchmark mergesort(question)

question = [3, 1, 4, 2], result = [1.0, 2.0, 3.0, 4.0]
question = [5, 1, 3, 9, 0], result = [0.0, 1.0, 3.0, 5.0, 9.0]


BenchmarkTools.Trial: 
  memory estimate:  1.50 KiB
  allocs estimate:  19
  --------------
  minimum time:     678.150 ns (0.00% GC)
  median time:      895.025 ns (0.00% GC)
  mean time:        1.044 μs (13.01% GC)
  maximum time:     19.126 μs (93.37% GC)
  --------------
  samples:          10000
  evals/sample:     160

## Python hand code

In [20]:
using PyCall
import PyCall
#this function works the same as the Julia one.
py"""
def merge(left, right):
    if not len(left) or not len(right):
        return left or right

    result = []
    i, j = 0, 0
    while (len(result) < len(left) + len(right)):
        if left[i] < right[j]:
            result.append(left[i])
            i+= 1
        else:
            result.append(right[j])
            j+= 1
        if i == len(left) or j == len(right):
            result.extend(left[i:] or right[j:])
            break 

    return result

def mergesort(list):
    if len(list) < 2:
        return list

    middle = len(list)/2
    left = mergesort(list[:middle])
    right = mergesort(list[middle:])

    return merge(left, right)
"""
pyMerge= py"mergesort"

PyObject <function mergesort at 0x7efc686796e0>

In [19]:
pyMerge([1,2,3,2,3,4,5,39,6,1])

10-element Array{Int64,1}:
  1
  1
  2
  2
  3
  3
  4
  5
  6
 39

## Benchmark and compare the mergesort

In [5]:
list1 = rand(10^1)


10-element Array{Float64,1}:
 0.532095
 0.702998
 0.474587
 0.480696
 0.918217
 0.408147
 0.242646
 0.521129
 0.265137
 0.91519 

In [6]:
list2 = rand(10^2)


100-element Array{Float64,1}:
 0.878333 
 0.670572 
 0.178788 
 0.499995 
 0.764966 
 0.58597  
 0.550583 
 0.0309895
 0.38604  
 0.524821 
 0.260489 
 0.990628 
 0.484277 
 ⋮        
 0.498048 
 0.427792 
 0.738624 
 0.669685 
 0.0474331
 0.513834 
 0.787073 
 0.116324 
 0.399483 
 0.219259 
 0.915294 
 0.231196 

In [7]:
list3 = rand(10^3)

1000-element Array{Float64,1}:
 0.906459  
 0.205111  
 0.695981  
 0.297911  
 0.446215  
 0.80322   
 0.144272  
 0.398349  
 0.651     
 0.787362  
 0.657078  
 0.369036  
 0.99056   
 ⋮         
 0.0222591 
 0.562864  
 0.492234  
 0.60971   
 0.586979  
 0.762661  
 0.526683  
 0.650137  
 0.312079  
 0.180246  
 0.0498706 
 0.00790802

In [8]:
list4 = rand(10^4)


10000-element Array{Float64,1}:
 0.925677  
 0.369049  
 0.171023  
 0.677286  
 0.246453  
 0.350996  
 0.199358  
 0.733291  
 0.0771156 
 0.386293  
 0.320286  
 0.298294  
 0.855126  
 ⋮         
 0.526782  
 0.999233  
 0.34051   
 0.948248  
 0.53551   
 0.865398  
 0.710215  
 0.00823558
 0.818313  
 0.804083  
 0.20132   
 0.794447  

In [9]:
list5 = rand(10^5)


100000-element Array{Float64,1}:
 0.413201 
 0.534188 
 0.243745 
 0.219544 
 0.645371 
 0.660504 
 0.25825  
 0.0989272
 0.489048 
 0.269433 
 0.712647 
 0.471524 
 0.018893 
 ⋮        
 0.434027 
 0.342826 
 0.848731 
 0.232476 
 0.693315 
 0.332053 
 0.0839995
 0.845872 
 0.963143 
 0.251643 
 0.429421 
 0.763278 

In [10]:
list6 = rand(10^6)


1000000-element Array{Float64,1}:
 0.421302 
 0.805541 
 0.065649 
 0.733359 
 0.680776 
 0.863403 
 0.912671 
 0.227805 
 0.866056 
 0.311765 
 0.144431 
 0.694784 
 0.934077 
 ⋮        
 0.452874 
 0.324267 
 0.558644 
 0.983651 
 0.925677 
 0.0129237
 0.517849 
 0.656013 
 0.436855 
 0.732021 
 0.252444 
 0.205842 

In [11]:
list7 = rand(10^7)

10000000-element Array{Float64,1}:
 0.0490567
 0.605519 
 0.796321 
 0.743196 
 0.902136 
 0.205133 
 0.631686 
 0.389377 
 0.821074 
 0.230321 
 0.990929 
 0.430442 
 0.506281 
 ⋮        
 0.5897   
 0.0685921
 0.130869 
 0.277318 
 0.526219 
 0.997023 
 0.120361 
 0.749436 
 0.474217 
 0.427984 
 0.859381 
 0.517562 

In [34]:
@benchmark sort(list1)

BenchmarkTools.Trial: 
  memory estimate:  240 bytes
  allocs estimate:  2
  --------------
  minimum time:     173.954 ns (0.00% GC)
  median time:      217.646 ns (0.00% GC)
  mean time:        233.522 ns (5.71% GC)
  maximum time:     7.188 μs (96.69% GC)
  --------------
  samples:          10000
  evals/sample:     737

In [35]:
@benchmark sort(list2)

BenchmarkTools.Trial: 
  memory estimate:  976 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.030 μs (0.00% GC)
  median time:      1.300 μs (0.00% GC)
  mean time:        1.526 μs (3.87% GC)
  maximum time:     113.023 μs (95.65% GC)
  --------------
  samples:          10000
  evals/sample:     20

In [36]:
@benchmark sort(list3)

BenchmarkTools.Trial: 
  memory estimate:  8.02 KiB
  allocs estimate:  2
  --------------
  minimum time:     17.900 μs (0.00% GC)
  median time:      23.901 μs (0.00% GC)
  mean time:        24.586 μs (0.00% GC)
  maximum time:     461.214 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [37]:
@benchmark sort(list4)

BenchmarkTools.Trial: 
  memory estimate:  78.28 KiB
  allocs estimate:  3
  --------------
  minimum time:     477.614 μs (0.00% GC)
  median time:      616.968 μs (0.00% GC)
  mean time:        628.770 μs (0.51% GC)
  maximum time:     100.736 ms (0.00% GC)
  --------------
  samples:          7828
  evals/sample:     1

In [38]:
@benchmark sort(list5)

BenchmarkTools.Trial: 
  memory estimate:  781.41 KiB
  allocs estimate:  3
  --------------
  minimum time:     5.872 ms (0.00% GC)
  median time:      7.018 ms (0.00% GC)
  mean time:        7.325 ms (0.54% GC)
  maximum time:     107.059 ms (0.00% GC)
  --------------
  samples:          682
  evals/sample:     1

In [39]:
@benchmark sort(list6)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  3
  --------------
  minimum time:     77.227 ms (0.00% GC)
  median time:      79.499 ms (0.00% GC)
  mean time:        83.601 ms (0.35% GC)
  maximum time:     105.641 ms (0.88% GC)
  --------------
  samples:          60
  evals/sample:     1

In [40]:
@benchmark sort(list7)

BenchmarkTools.Trial: 
  memory estimate:  76.29 MiB
  allocs estimate:  3
  --------------
  minimum time:     934.552 ms (0.03% GC)
  median time:      1.013 s (1.09% GC)
  mean time:        1.003 s (3.21% GC)
  maximum time:     1.069 s (6.55% GC)
  --------------
  samples:          5
  evals/sample:     1

In [41]:
@benchmark $PyCall.sort(list1)

BenchmarkTools.Trial: 
  memory estimate:  240 bytes
  allocs estimate:  2
  --------------
  minimum time:     231.681 ns (0.00% GC)
  median time:      271.502 ns (0.00% GC)
  mean time:        298.281 ns (4.07% GC)
  maximum time:     9.496 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     442

In [42]:
@benchmark $PyCall.sort(list2)

BenchmarkTools.Trial: 
  memory estimate:  976 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.170 μs (0.00% GC)
  median time:      1.350 μs (0.00% GC)
  mean time:        1.636 μs (2.83% GC)
  maximum time:     149.154 μs (95.79% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [43]:
@benchmark $PyCall.sort(list3)

BenchmarkTools.Trial: 
  memory estimate:  8.02 KiB
  allocs estimate:  2
  --------------
  minimum time:     18.000 μs (0.00% GC)
  median time:      20.400 μs (0.00% GC)
  mean time:        22.418 μs (0.00% GC)
  maximum time:     448.214 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [44]:
@benchmark $PyCall.sort(list4)

BenchmarkTools.Trial: 
  memory estimate:  78.28 KiB
  allocs estimate:  3
  --------------
  minimum time:     476.014 μs (0.00% GC)
  median time:      567.567 μs (0.00% GC)
  mean time:        590.478 μs (0.58% GC)
  maximum time:     8.888 ms (0.00% GC)
  --------------
  samples:          8332
  evals/sample:     1

In [45]:
@benchmark $PyCall.sort(list5)

BenchmarkTools.Trial: 
  memory estimate:  781.41 KiB
  allocs estimate:  3
  --------------
  minimum time:     6.338 ms (0.00% GC)
  median time:      6.741 ms (0.00% GC)
  mean time:        7.055 ms (0.42% GC)
  maximum time:     14.626 ms (0.00% GC)
  --------------
  samples:          708
  evals/sample:     1

In [46]:
@benchmark $PyCall.sort(list6)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  3
  --------------
  minimum time:     76.959 ms (0.00% GC)
  median time:      83.872 ms (0.00% GC)
  mean time:        85.233 ms (0.43% GC)
  maximum time:     96.829 ms (0.00% GC)
  --------------
  samples:          59
  evals/sample:     1

In [47]:
@benchmark $PyCall.sort(list7)

BenchmarkTools.Trial: 
  memory estimate:  76.29 MiB
  allocs estimate:  3
  --------------
  minimum time:     962.694 ms (1.31% GC)
  median time:      1.125 s (1.12% GC)
  mean time:        1.094 s (3.20% GC)
  maximum time:     1.259 s (6.12% GC)
  --------------
  samples:          5
  evals/sample:     1

In [12]:
@benchmark mergesort(list1)

BenchmarkTools.Trial: 
  memory estimate:  4.58 KiB
  allocs estimate:  67
  --------------
  minimum time:     1.990 μs (0.00% GC)
  median time:      2.970 μs (0.00% GC)
  mean time:        3.800 μs (14.60% GC)
  maximum time:     339.760 μs (96.37% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [13]:
@benchmark mergesort(list2)

BenchmarkTools.Trial: 
  memory estimate:  93.77 KiB
  allocs estimate:  1378
  --------------
  minimum time:     41.702 μs (0.00% GC)
  median time:      50.202 μs (0.00% GC)
  mean time:        64.621 μs (15.34% GC)
  maximum time:     2.930 ms (96.37% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [14]:
@benchmark mergesort(list3)

BenchmarkTools.Trial: 
  memory estimate:  1.26 MiB
  allocs estimate:  19487
  --------------
  minimum time:     564.116 μs (0.00% GC)
  median time:      685.221 μs (0.00% GC)
  mean time:        849.684 μs (15.65% GC)
  maximum time:     15.948 ms (88.60% GC)
  --------------
  samples:          5818
  evals/sample:     1

In [15]:
@benchmark mergesort(list4)

BenchmarkTools.Trial: 
  memory estimate:  18.57 MiB
  allocs estimate:  270070
  --------------
  minimum time:     8.876 ms (0.00% GC)
  median time:      11.823 ms (19.07% GC)
  mean time:        12.035 ms (16.68% GC)
  maximum time:     20.085 ms (15.60% GC)
  --------------
  samples:          415
  evals/sample:     1

In [16]:
@benchmark mergesort(list5)

BenchmarkTools.Trial: 
  memory estimate:  227.46 MiB
  allocs estimate:  3396440
  --------------
  minimum time:     135.350 ms (15.78% GC)
  median time:      144.412 ms (16.82% GC)
  mean time:        155.669 ms (20.74% GC)
  maximum time:     261.183 ms (48.18% GC)
  --------------
  samples:          33
  evals/sample:     1

In [17]:
@benchmark mergesort(list6)

BenchmarkTools.Trial: 
  memory estimate:  2.49 GiB
  allocs estimate:  39543251
  --------------
  minimum time:     1.761 s (18.87% GC)
  median time:      2.005 s (20.64% GC)
  mean time:        1.924 s (20.10% GC)
  maximum time:     2.007 s (20.63% GC)
  --------------
  samples:          3
  evals/sample:     1

In [18]:
@benchmark mergesort(list7)

BenchmarkTools.Trial: 
  memory estimate:  30.55 GiB
  allocs estimate:  468751252
  --------------
  minimum time:     25.344 s (26.32% GC)
  median time:      25.344 s (26.32% GC)
  mean time:        25.344 s (26.32% GC)
  maximum time:     25.344 s (26.32% GC)
  --------------
  samples:          1
  evals/sample:     1

In [55]:
@benchmark pyMerge(list1)

BenchmarkTools.Trial: 
  memory estimate:  3.58 KiB
  allocs estimate:  100
  --------------
  minimum time:     283.508 μs (0.00% GC)
  median time:      363.910 μs (0.00% GC)
  mean time:        368.655 μs (0.40% GC)
  maximum time:     32.571 ms (45.50% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [56]:
@benchmark pyMerge(list2)

BenchmarkTools.Trial: 
  memory estimate:  26.11 KiB
  allocs estimate:  730
  --------------
  minimum time:     1.126 ms (0.00% GC)
  median time:      1.313 ms (0.00% GC)
  mean time:        1.446 ms (1.70% GC)
  maximum time:     46.954 ms (46.33% GC)
  --------------
  samples:          3438
  evals/sample:     1

In [57]:
@benchmark pyMerge(list3)

BenchmarkTools.Trial: 
  memory estimate:  251.17 KiB
  allocs estimate:  7030
  --------------
  minimum time:     11.302 ms (0.00% GC)
  median time:      12.500 ms (0.00% GC)
  mean time:        13.758 ms (3.70% GC)
  maximum time:     158.194 ms (77.43% GC)
  --------------
  samples:          364
  evals/sample:     1

In [58]:
@benchmark pyMerge(list4)

BenchmarkTools.Trial: 
  memory estimate:  2.44 MiB
  allocs estimate:  70032
  --------------
  minimum time:     128.765 ms (0.00% GC)
  median time:      140.320 ms (0.00% GC)
  mean time:        145.159 ms (1.33% GC)
  maximum time:     210.442 ms (11.42% GC)
  --------------
  samples:          35
  evals/sample:     1

In [59]:
@benchmark pyMerge(list5)

BenchmarkTools.Trial: 
  memory estimate:  24.42 MiB
  allocs estimate:  700034
  --------------
  minimum time:     1.544 s (1.30% GC)
  median time:      1.599 s (1.27% GC)
  mean time:        1.598 s (1.41% GC)
  maximum time:     1.649 s (1.78% GC)
  --------------
  samples:          4
  evals/sample:     1

In [60]:
@benchmark pyMerge(list6)

BenchmarkTools.Trial: 
  memory estimate:  244.15 MiB
  allocs estimate:  7000044
  --------------
  minimum time:     18.893 s (1.83% GC)
  median time:      18.893 s (1.83% GC)
  mean time:        18.893 s (1.83% GC)
  maximum time:     18.893 s (1.83% GC)
  --------------
  samples:          1
  evals/sample:     1

In [ ]:
@benchmark pyMerge(list7)

## Comparison between sort() methods
We can see from the test results above that the julia built in sort function is the fastest one, and most of the cases can be done in ns or us. The speed of Julia implementation of Python built in is smiliar to the Julia built in code, but it's a little bit slower.Julia hand code is a little bit slower than Python built in, but again it's not a very big difference. But the Python hand code is far slower than the other three codes, especially when the data size is very large. It's about 10^3 or 10^4 times slower.


## Python sorted() exploration
https://github.com/python/cpython/blob/master/Objects/listobject.c   
From line 1089 to line 2399 are all the source code (in C language) for sorted () in python.
Since there we have about 1300 lines for the source code, I don’t think there’s necessity to copy them here.   
The algorithm used in sorted is known as Timesort, in which the code break the data up into blocks, pre-sort those blocks using insertion sort, and then merge the blocks together using mergesort. “In early python-versions, the sort function implemented a modified version of quicksort. However, it was deemed unstable and as of 2.3 they switched to using an adaptive mergesort algorithm.”    
(https://stackoverflow.com/questions/1517347/about-pythons-built-in-sort-method)   
But also in the source code, I found it also has the code for binarysort, so maybe the code can execute different algorithms on inputs with different sizes.   

https://github.com/python/cpython/blob/master/Objects/listsort.txt   
Here’s a comparisons of the merge sort and the Python's Samplesort Hybrid. In this experiment the author run the two algorithms on different kind of lists. Here’re some lists he run on:
1. *sort: random data
2. \sort: descending data
3. /sort: ascending data
4. 3sort: ascending, then 3 random exchanges
5. +sort: ascending, then 10 random at the end
6. %sort: ascending, then randomly replace 1% of elements w/ random values
7. ~sort: many duplicates
8. =sort: all equal
9. !sort: worst case scenario   

The algorithms are “effectively identical” in the cases 1 to 3, with a large variety on the efficiency for the following cases

## Julia sort() exploration
Cited from https://docs.julialang.org/en/stable/stdlib/sort/#Sorting-Algorithms-1

#### The mechanism by which Julia picks default sorting algorithms 
- is implemented via the Base.Sort.defalg function. 
- allows a particular algorithm to be registered as the default in all sorting functions for specific arrays. For example, here are the two default methods from sort.jl:
```
defalg(v::AbstractArray) = MergeSort
defalg{T<:Number}(v::AbstractArray{T}) = QuickSort
```
- can specify an algorithm to use via the `alg` keyword 
- acted as a variant of the function `sort!` which is written in Julia     
For example:
```
function sort!(v::AbstractVector, lo::Int, hi::Int, a::MergeSortAlg, o::Ordering, t=similar(v,0))
    @inbounds if lo < hi
        hi-lo <= SMALL_THRESHOLD && return sort!(v, lo, hi, SMALL_ALGORITHM, o)

        m = (lo+hi)>>>1
        (length(t) < m-lo+1) && resize!(t, m-lo+1)

        sort!(v, lo,  m,  a, o, t)
        sort!(v, m+1, hi, a, o, t)

        i, j = 1, lo
        while j <= m
            t[i] = v[j]
            i += 1
            j += 1
        end

        i, k = 1, lo
        while k < j <= hi
            if lt(o, v[j], t[i])
                v[k] = v[j]
                j += 1
            else
                v[k] = t[i]
                i += 1
            end
            k += 1
        end
        while k < j
            v[k] = t[i]
            k += 1
            i += 1
        end
    end

    return v
end